In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import pickle

import tensorflow as tf
from tensorflow.keras import layers, regularizers, Input, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.mixed_precision import set_global_policy

In [2]:
set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Laptop GPU, compute capability 8.6


In [3]:
train_df = pd.read_csv(r"S:\Coding\Python\MiniProject\Dataset\80_20excel\train.csv")
test_df = pd.read_csv(r"S:\Coding\Python\MiniProject\Dataset\80_20excel\test.csv")

In [4]:
le = LabelEncoder()
train_df['gender_encoded'] = le.fit_transform(train_df['gender'])
test_df['gender_encoded'] = le.transform(test_df['gender'])

In [5]:
IMG_SIZE = (128,128)

In [6]:
def load_images(image_dir, ids):
    images = []
    for img_id in tqdm(ids):
        img_path = os.path.join(image_dir, str(img_id) + ".png")
        image = Image.open(img_path).convert('RGB')
        image = image.resize(IMG_SIZE)
        image = img_to_array(image)
        image = preprocess_input(image)
        images.append(image)
    return np.array(images)

In [7]:
train_images = load_images(r"S:\Coding\Python\MiniProject\Dataset\80_20image\train", train_df['id'])
test_images = load_images(r"S:\Coding\Python\MiniProject\Dataset\80_20image\test", test_df['id'])

 70%|██████████████████████████████████████████████████████                       | 7080/10089 [05:24<02:18, 21.79it/s]

KeyboardInterrupt



In [ ]:
train_gender = train_df['gender_encoded'].values
test_gender = test_df['gender_encoded'].values

In [9]:
y_train = train_df['year'].values
y_test = test_df['year'].values

In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense, Concatenate,
                                     Multiply, Add, Activation, Conv2D, Reshape)
from tensorflow.keras.models import Model
import tensorflow as tf

def cbam(x, ratio=16, kernel_size=7):
    channel = x.shape[-1]

    shared_dense1 = Dense(channel // ratio, activation='relu')
    shared_dense2 = Dense(channel)

    avg_pool = GlobalAveragePooling2D()(x)
    max_pool = tf.reduce_max(x, axis=[1, 2])

    avg_pool = Reshape((1, 1, channel))(avg_pool)
    max_pool = Reshape((1, 1, channel))(max_pool)

    avg_out = shared_dense2(shared_dense1(avg_pool))
    max_out = shared_dense2(shared_dense1(max_pool))

    channel_attention = Activation('sigmoid')(Add()([avg_out, max_out]))
    channel_refined = Multiply()([x, channel_attention])

    avg_spatial = tf.reduce_mean(channel_refined, axis=-1, keepdims=True)
    max_spatial = tf.reduce_max(channel_refined, axis=-1, keepdims=True)

    spatial_attention = Conv2D(1, kernel_size, padding='same', activation='sigmoid')(
        Concatenate()([avg_spatial, max_spatial])
    )

    refined = Multiply()([channel_refined, spatial_attention])
    return refined

image_input = Input(shape=(128, 128, 3))
base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=image_input)

x = base_model.output
x = cbam(x)
x = GlobalAveragePooling2D()(x)

meta_input = Input(shape=(1,))
x = Concatenate()([x, meta_input])

x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1)(x)

model = Model(inputs=[image_input, meta_input], outputs=output)
model.compile(optimizer='adam', loss='mae', metrics=['mae'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [11]:
checkpoint_cb = ModelCheckpoint(
    filepath=r'S:\Coding\Python\MiniProject\Models\HybridModel2\epoch_{epoch:02d}_valmae_{val_mae:.2f}.h5',
    save_freq='epoch',
    save_weights_only=False,
    save_best_only=False,
    verbose=1
)
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [12]:
history = model.fit(
    [train_images, train_gender], y_train,
    validation_data=([test_images, test_gender], y_test),
    epochs=40,
    batch_size=8,
    callbacks=[early_stop, lr_scheduler, checkpoint_cb]
)

Epoch 1/40
1262/1262 [==============================] - ETA: 0s - loss: 21.8743 - mae: 21.8742       
Epoch 1: saving model to S:\Coding\Python\MiniProject\Models\HybridModel2\epoch_01_valmae_121.45.h5
1262/1262 [==============================] - 186s 137ms/step - loss: 21.8743 - mae: 21.8742 - val_loss: 121.4531 - val_mae: 121.4505 - lr: 0.0010
Epoch 2/40
1262/1262 [==============================] - ETA: 0s - loss: 15.3332 - mae: 15.3333  
Epoch 2: saving model to S:\Coding\Python\MiniProject\Models\HybridModel2\epoch_02_valmae_19.30.h5
1262/1262 [==============================] - 172s 136ms/step - loss: 15.3332 - mae: 15.3333 - val_loss: 19.2961 - val_mae: 19.2963 - lr: 0.0010
Epoch 3/40
1262/1262 [==============================] - ETA: 0s - loss: 13.1944 - mae: 13.1944  
Epoch 3: saving model to S:\Coding\Python\MiniProject\Models\HybridModel2\epoch_03_valmae_48.42.h5
1262/1262 [==============================] - 186s 148ms/step - loss: 13.1944 - mae: 13.1944 - val_loss: 48.4145 - va

In [13]:
model.save(r'S:\Coding\Python\MiniProject\Models\HybridModel2')
model.save(r'S:\Coding\Python\MiniProject\Models\HybridModel2.h5')

INFO:tensorflow:Assets written to: S:\Coding\Python\MiniProject\Models\HybridModel2\assets


INFO:tensorflow:Assets written to: S:\Coding\Python\MiniProject\Models\HybridModel2\assets


In [14]:
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print("Training complete. History saved as 'training_history.pkl'")

Training complete. History saved as 'training_history.pkl'
